# Yield estimation with PlanetScope

NOTE: This notebook uses imagery that was generously provided by Planet through their Education and Research program to Grady Killeen (via affiliation with Berkeley). Planet support verified that it was within the terms of the program to use imagery from this program to train researchers at PAD to use the imagery, even though imagery is not available through this program to PAD. The Sentinel-2 imagery can be shared upon request, but the PlanetScope imagery used in the notebook cannot be provided to ensure that Planet's terms of service are being respected. Anyone at PAD that uses to wish PlanetScope imagery should reach out to Planet about purchasing the imagery.

Sometimes Sentinel-2 imagery has inadequate resolution, or there are no cloud free images available during a date range. In these cases, PlanetScope imagery may offer a solution. It offers imagery at a 3.7m by 3.7m meter resolution and includes red, green, blue, and near-infrared bands. The constellation has a daily revisit rate, so the chances of obtaining cloud-free images are about 5 times higher than in the case of Sentinel-2. 

Planet is a private company, and its imagery is typically sold. However, they offer an [Education and Research Program](https://www.planet.com/markets/education-and-research/) that includes access to 5,000 km^2 of PlanetScope imagery per month at no cost, with the condition that this imagery is used for academic research. By efficiently clipping imagery to an AOI, this quota is sufficient for effective crop yield estimation. NGO and government employees are not eligible for this program, even if they are working on academic research.

To create the environment, first install geopandas, gdal, fiona, shapely, pyproj, and Planet's API (package name `planet`), ideally from conda-forge where possible. Then activate the environment and enter `planet init` from a command line to enter your username and password generated from Planet.com. This should ensure that all of your API requests are properly authenticated. 

This tutorial uses code based on the [porder](https://samapriya.github.io/projects/porder/) package, which is an excellent command line tool for downloading Planet imagery. Some of the core functions were modified to work from this notebook. It also uses code modified from one of Planet's own notebooks.

## Preparing the AOI information

If we try to download every scene contained in the AOI, the quota would quickly run out. Instead, we will want to download images that are clipped to only include the area of each plot plus a small additional buffer. The Planet API offers a clip option that handles this for us, and only the clipped imagery is counted against the quota. This makes it feasible to download multiple dates of imagery for the sample without exceeding the quota. 

To achieve this, we will need to import the plot boundary data, take a small buffer around each plot, and take an envelope of the vector data (the set of smallest rectangles containing each polygon). The Planet API supports a maximum of 500 vertices in the GeoJSON that's passed, so we will break the full set of rectangles into multiple lists containing no more than 100 plots each. We will then flatten the GeoJSON into the format required by the API before searching for imagery. 

In [1]:
import geopandas as gpd 
import folium


# Load the plot boundary data 
gdf = gpd.read_file('uganda-crops/merged/all_plots.geojson')
gdf.crs = 'epsg:4326'

# Buffer
buffered = gdf.copy(deep=True)
buffered = buffered.to_crs('esri:54031')  # Equidistant projection
buffered['geometry'] = buffered['geometry'].buffer(10)  
buffered = buffered.to_crs('epsg:4326')

# Generate envelope 
envelope = buffered.copy(deep=True)
envelope['geometry'] = envelope['geometry'].envelope

# Plot the original data and envelope in Folium to check for issues 
envelope['lat'] = envelope.geometry.centroid.y
envelope['lon'] = envelope.geometry.centroid.x
m = folium.Map([envelope['lat'].mean(), envelope['lon'].mean()], zoom_start=12, tiles='Stamen Terrain')
folium.GeoJson(gdf, name='Sample plots').add_to(m)
folium.GeoJson(envelope, name='Envelope').add_to(m)
m

/home/grady/anaconda3/envs/porder/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/grady/anaconda3/envs/porder/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



The envelopes look correct. Hence, we will format them for the Planet API, and then split them into lists of 100. We will create 3 geojson files that contain only a single feature, which is a multipolygon containing the envelopes for each plot. This format seems to work well with the Planet API.

In [2]:
import os 
from shapely.geometry import * 
from geojson import Feature, FeatureCollection, dump


# Generate path for Planet shapefiles if it doesn't exist 
if not os.path.isdir('uganda-crops/planet'):
    os.mkdir('uganda-crops/planet')

# Partition full plot boundary data set into GDFs of max length 100 
planet_gdf_1 = envelope.iloc[:100]
planet_gdf_2 = envelope.iloc[100:200]
planet_gdf_3 = envelope.iloc[200:]

# Convert each GDF to a flattened geojson (single multipolygon) then save 
i = 1
for x in [planet_gdf_1, planet_gdf_2, planet_gdf_3]:
    flattened = x.geometry.unary_union
    multi = MultiPolygon(flattened)
    features = []
    features.append(Feature(geometry=multi, properties={"ID": "1"}))
    feature_collection = FeatureCollection(features)
    out_path = 'uganda-crops/planet/aoi_{}.geojson'.format(i)
    with open(out_path, 'w') as out:
        dump(feature_collection, out)
    i += 1

## Filter and download PlanetScope data

Now that we have the properly formatted AOIs, we will call the Planet API to filter for relevant PlanetScope scenes and then download the imagery. To filter the scenes, we will use code adapted from the porder package. To download the scenes, we will use code adapted from one of Planet's own training notebooks. Since this is just a demo and PlanetScope data is acquired more frequently, we will only download data from 2017-09-23. Inpsection of the AOI on Planet's website indicated that low cloud imagery covering the majority of the sample is available on this date. 

In general, it is useful to use [Planet's website](https://www.planet.com/explorer/) to identify appropriate dates before using the API. The website lists the total coverage of all images meeting the filtering criteria for a given date. This functionality is useful since multiple PlanetScope images need to be stitched together to cover a large portion of the sample.

We will query and download images separately for each AOI so the geojson object has a small enough number of vertices and we meet the AOI rate limits. We will download the 4 band analytic surface reflectance product for each date. These images include red, green, blue, and near-infrared surface reflactance values.

### AOI 1

#### Search for images

In [3]:
import requests 
import json
import pyproj
from planet import api
from planet.api import filters
from planet.api.auth import find_api_key
from shapely.geometry import shape
from shapely.ops import transform


# Load Planet API key and initialize a session
API_KEY = find_api_key()
client = api.ClientV1(API_KEY)

# Define a function to search for IDs in the AOI taken from the porder package 
stbase={'config': [], 'field_name': [], 'type': 'StringInFilter'}
rbase={'config': {'gte': [], 'lte': []},'field_name': [], 'type': 'RangeFilter'}

# Area lists
ar = []
far = []
ovall=[]

# Function to use the client and then search
def idl(**kwargs):
    _id_list = []
    for key,value in kwargs.items():
        if key=='infile' and value is not None:
            infile=value
            try:
                with open(infile) as f:
                    aoi = json.load(f)
                temp = aoi['features'][0]['geometry']
            except Exception as e:
                print('Could not parse geometry')
                print(e)
        if key=='item' and value is not None:
            try:
                item=value
            except Exception as e:
                sys.exit(e)
        if key=='start' and value is not None:
            try:
                start=value
                st = filters.date_range('acquired', gte=start)
            except Excpetion as e:
                sys.exit(e)
        if key=='end' and value is not None:
            end=value
            ed=filters.date_range('acquired', lte=end)
        if key == 'asset' and value  is not None:
            try:
                asset=value
            except Exception as e:
                sys.exit(e)
        if key == 'cmin':
            if value ==None:
                try:
                    cmin=0
                except Exception as e:
                    print(e)
            if value  is not None:
                try:
                    cmin=float(value)
                except Exception as e:
                    print(e)
        if key == 'cmax':
            if value ==None:
                try:
                    cmax=1
                except Exception as e:
                    print(e)
            elif value  is not None:
                try:
                    cmax=float(value)
                except Exception as e:
                    print(e)
        if key=='num':
            if value is not None:
                num=value
            elif value==None:
                num=1000000
        if key == 'ovp':
            if value is not None:
                ovp=int(value)
            elif value == None:
                ovp=0.01
        if key== 'filters' and value is not None:
            for items in value:
                ftype=items.split(':')[0]
                if ftype=='string':
                    try:
                        fname=items.split(':')[1]
                        fval=items.split(':')[2]
                        #stbase={'config': [], 'field_name': [], 'type': 'StringInFilter'}
                        stbase['config']=fval.split(',')#fval
                        stbase['field_name']=fname
                    except Exception as e:
                        print(e)
                elif ftype=='range':
                    fname=items.split(':')[1]
                    fgt=items.split(':')[2]
                    flt=items.split(':')[3]
                    #rbase={'config': {'gte': [], 'lte': []},'field_name': [], 'type': 'RangeFilter'}
                    rbase['config']['gte']=int(fgt)
                    rbase['config']['lte']=int(flt)
                    rbase['field_name']=fname

    print('Running search for a maximum of: ' + str(num) + ' assets')
    l=0
    sgeom=filters.geom_filter(temp)
    aoi_shape = shape(temp)
    if not aoi_shape.is_valid:
        aoi_shape=aoi_shape.buffer(0)
        #print('Your Input Geometry is invalid & may have issues:A valid Polygon may not possess anyoverlapping exterior or interior rings.'+'\n')
    date_filter = filters.date_range('acquired', gte=start,lte=end)
    cloud_filter = filters.range_filter('cloud_cover', gte=cmin,lte=cmax)
    asset_filter=filters.permission_filter('assets.'+str(asset.split(',')[0])+':download')
    # print(rbase)
    # print(stbase)
    if len(rbase['field_name']) !=0 and len(stbase['field_name']) !=0:
        and_filter = filters.and_filter(date_filter, cloud_filter,asset_filter,sgeom,stbase,rbase)
    elif len(rbase['field_name']) ==0 and len(stbase['field_name']) !=0:
        and_filter = filters.and_filter(date_filter, cloud_filter,asset_filter,sgeom,stbase)
    elif len(rbase['field_name']) !=0 and len(stbase['field_name']) ==0:
        and_filter = filters.and_filter(date_filter, cloud_filter,asset_filter,sgeom,rbase)
    elif len(rbase['field_name']) ==0 and len(stbase['field_name'])==0:
        and_filter = filters.and_filter(date_filter, cloud_filter,asset_filter,sgeom)
    item_types = [item]
    req = filters.build_search_request(and_filter, item_types)
    res = client.quick_search(req)
    for things in res.items_iter(1000000): # A large number as max number to check against
        try:
            all_assets=[assets.split(':')[0].replace('assets.','') for assets in things['_permissions']]
            if things['properties']['quality_category'] =='standard' and all(elem in all_assets for elem in asset.split(',')):
                itemid=things['id']
                footprint = things["geometry"]
                s = shape(footprint)
                if item.startswith('SkySat'):
                    epsgcode='3857'
                else:
                    epsgcode=things['properties']['epsg_code']
                if aoi_shape.area>s.area:
                    intersect=(s).intersection(aoi_shape)
                elif s.area>=aoi_shape.area:
                    intersect=(aoi_shape).intersection(s)
                proj_transform = pyproj.Transformer.from_proj(pyproj.Proj(4326), pyproj.Proj(epsgcode), always_xy=True).transform # always_xy determines correct coord order
                print('Processing ' + str(len(ar) + 1) + ' items with total area '+ str("{:,}".format(round(sum(far)))) + ' sqkm', end='\r')
                if transform(proj_transform, (aoi_shape)).area>transform(proj_transform,s).area:
                    if (transform(proj_transform, intersect).area / transform(proj_transform, s).area*100)>=ovp:
                        ar.append(transform(proj_transform, intersect).area/1000000)
                        far.append(transform(proj_transform, s).area/1000000)
                        _id_list.append(itemid)
                elif transform(proj_transform, s).area>=transform(proj_transform, aoi_shape).area:
                    if (transform(proj_transform, intersect).area/transform(proj_transform, aoi_shape).area*100)>=ovp:
                        ar.append(transform(proj_transform, intersect).area/1000000)
                        far.append(transform(proj_transform, s).area/1000000)
                        _id_list.append(itemid)
            if int(len(ar))==int(num):
                break
        except Exception as e:
            pass
    print('Total estimated cost to quota: ' + str("{:,}".format(round(sum(far)))) + ' sqkm')
    print('Total estimated cost to quota if clipped: ' + str("{:,}".format(round(sum(ar)))) + ' sqkm')
    return _id_list
    
# Filter for imagery
if not os.path.isfile('uganda-crops/locks/aoi-1'):  # Skip if already downloaded
    aoi_1_ids = idl(infile='uganda-crops/planet/aoi_1.geojson', item='PSScene4Band', asset='analytic',
                    cmin=0.0, cmax=0.1, start='2017-09-23', end='2017-09-24', ovp=1, num=35)

else:
    print('AOI 1 already downloaded.')

AOI 1 already downloaded.


#### Download images 

In [4]:
from requests.auth import HTTPBasicAuth
import pathlib
import time


# Create folders to store the imagery in if they don't exist
if not os.path.isdir('imagery/planet'):
    os.mkdir('imagery/planet')
    
if not os.path.isdir('imagery/planet/aoi-1'):
    os.mkdir('imagery/planet/aoi-1')
    
if not os.path.isdir('imagery/planet/aoi-2'):
    os.mkdir('imagery/planet/aoi-2')
    
if not os.path.isdir('imagery/planet/aoi-3'):
    os.mkdir('imagery/planet/aoi-3')

# Create a folder to add file locks to 
if not os.path.isdir('uganda-crops/locks'):
    os.mkdir('uganda-crops/locks')
    
# Add in code adapted from one of Planet's example ordering notebooks
# set up requests to work with api
auth = HTTPBasicAuth(API_KEY, '')
headers = {'content-type': 'application/json'}
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# define helpful functions for submitting, polling, and downloading an order
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    
    if not response.ok:
        raise Exception(response.content)

    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

def poll_for_success(order_url, auth, num_loops=50):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        success_states = ['success', 'partial']
        if state == 'failed':
            raise Exception(response)
        elif state in success_states:
            break
        
        time.sleep(60)
        
def download_order(order_url, auth, overwrite=False, output_path='data'):
    r = requests.get(order_url, auth=auth)
    print(r)

    response = r.json()
    results = response['_links']['results']
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    results_paths = [pathlib.Path(os.path.join(output_path, n)) for n in results_names]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name, path in zip(results_urls, results_names, results_paths):
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
    return dict(zip(results_names, results_paths)) 
    
# Check if the lock for AOI 1 exists (meaning we already ordered it), if not create the lock and order the data 
if not os.path.isfile('uganda-crops/locks/aoi-1'):
    open('uganda-crops/locks/aoi-1', 'a').close()  # Create lock
    
    aoi_1_products = [
        {
          "item_ids": aoi_1_ids,
          "item_type": "PSScene4Band",
          "product_bundle": "analytic_sr"
        }
    ]

    with open('uganda-crops/planet/aoi_1.geojson') as f:
        aoi = json.load(f)
    clip_aoi = aoi['features'][0]['geometry']
    
    clip = {
        "clip": {
            "aoi": clip_aoi
        }
    }
    
    # Create order information 
    request_aoi_1 = {
      "name": "Uganda Training: AOI-1",
      "products": aoi_1_products,
      "tools": [clip]
    }
    
    order_url_aoi_1 = place_order(request_aoi_1, auth)
    poll_for_success(order_url_aoi_1, auth)
    time.sleep(30)  # Avoid 423 exception for too many requests 
    downloaded_files_aoi_1 = download_order(order_url_aoi_1, auth, False, 'imagery/planet/aoi-1')

else:
    print('Products already ordered')


Products already ordered


### AOI 2

In [5]:
# Search for images 
if not os.path.isfile('uganda-crops/locks/aoi-2'):  # Skip if already downloaded
    aoi_2_ids = idl(infile='uganda-crops/planet/aoi_2.geojson', item='PSScene4Band', asset='analytic',
                    cmin=0.0, cmax=0.1, start='2017-09-23', end='2017-09-24', ovp=1, num=35)

else:
    print('AOI 2 already downloaded.')
    
# Download images 
if not os.path.isfile('uganda-crops/locks/aoi-2'):
    open('uganda-crops/locks/aoi-1', 'a').close()  # Create lock
    
    aoi_2_products = [
        {
          "item_ids": aoi_2_ids,
          "item_type": "PSScene4Band",
          "product_bundle": "analytic_sr"
        }
    ]

    with open('uganda-crops/planet/aoi_2.geojson') as f:
        aoi = json.load(f)
    clip_aoi = aoi['features'][0]['geometry']
    
    clip = {
        "clip": {
            "aoi": clip_aoi
        }
    }
    
    # Create order information 
    request_aoi_2 = {
      "name": "Uganda Training: AOI-2",
      "products": aoi_2_products,
      "tools": [clip]
    }
    
    order_url_aoi_2 = place_order(request_aoi_2, auth)
    poll_for_success(order_url_aoi_2, auth)
    time.sleep(30)  # Avoid 423 exception for too many requests 
    downloaded_files_aoi_2 = download_order(order_url_aoi_2, auth, False, 'imagery/planet/aoi-2')

else:
    print('Products already ordered')

Running search for a maximum of: 35 assets
Total estimated cost to quota: 6,822 sqkm sqkm
Total estimated cost to quota if clipped: 1 sqkm
<Response [202]>
bf9dccc6-feb9-490c-b368-3bdc9ee3721f
queued
running
running
running
running
running
running
running
success
<Response [200]>
137 items to download
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072709_0f22_3B_AnalyticMS_SR_clip.tif to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072709_0f22_3B_AnalyticMS_SR_clip.tif
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072459_1014_3B_AnalyticMS_DN_udm_clip.tif to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072459_1014_3B_AnalyticMS_DN_udm_clip.tif
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072459_1014_3B_AnalyticMS_metadata_clip.xml to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072459_1014_3B_AnalyticMS_metad

downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072709_0f22_metadata.json to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072709_0f22_metadata.json
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072534_1035_3B_AnalyticMS_metadata_clip.xml to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072534_1035_3B_AnalyticMS_metadata_clip.xml
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072205_0f15_metadata.json to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072205_0f15_metadata.json
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072155_0f15_3B_AnalyticMS_SR_clip.tif to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072155_0f15_3B_AnalyticMS_SR_clip.tif
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072457_1014_3B_AnalyticMS_metadata_clip.xml t

downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_085620_101c_3B_AnalyticMS_DN_udm_clip.tif to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_085620_101c_3B_AnalyticMS_DN_udm_clip.tif
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_085620_101c_3B_AnalyticMS_SR_clip.tif to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_085620_101c_3B_AnalyticMS_SR_clip.tif
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_085907_0f21_3B_AnalyticMS_metadata_clip.xml to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_085907_0f21_3B_AnalyticMS_metadata_clip.xml
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_085901_0f21_3B_AnalyticMS_DN_udm_clip.tif to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_085901_0f21_3B_AnalyticMS_DN_udm_clip.tif
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSS

downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072545_1035_3B_AnalyticMS_SR_clip.tif to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072545_1035_3B_AnalyticMS_SR_clip.tif
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072545_1035_metadata.json to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072545_1035_metadata.json
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072545_1035_3B_AnalyticMS_DN_udm_clip.tif to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072545_1035_3B_AnalyticMS_DN_udm_clip.tif
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072545_1035_3B_AnalyticMS_metadata_clip.xml to imagery/planet/aoi-2/bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_072545_1035_3B_AnalyticMS_metadata_clip.xml
downloading bf9dccc6-feb9-490c-b368-3bdc9ee3721f/PSScene4Band/20170923_085903_0f21_3

### AOI 3

In [6]:
# Sleep for 5 minutes to avoid triggering rate limits 
time.sleep(300)

# Search for images 
if not os.path.isfile('uganda-crops/locks/aoi-3'):  # Skip if already downloaded
    aoi_3_ids = idl(infile='uganda-crops/planet/aoi_3.geojson', item='PSScene4Band', asset='analytic',
                    cmin=0.0, cmax=0.1, start='2017-09-23', end='2017-09-24', ovp=1, num=35)

else:
    print('AOI 3 already downloaded.')
    
# Download images 
if not os.path.isfile('uganda-crops/locks/aoi-3'):
    open('uganda-crops/locks/aoi-3', 'a').close()  # Create lock
    
    aoi_3_products = [
        {
          "item_ids": aoi_3_ids,
          "item_type": "PSScene4Band",
          "product_bundle": "analytic_sr"
        }
    ]

    with open('uganda-crops/planet/aoi_3.geojson') as f:
        aoi = json.load(f)
    clip_aoi = aoi['features'][0]['geometry']
    
    clip = {
        "clip": {
            "aoi": clip_aoi
        }
    }
    
    # Create order information 
    request_aoi_3 = {
      "name": "Uganda Training: AOI-3",
      "products": aoi_3_products,
      "tools": [clip]
    }
    
    order_url_aoi_3 = place_order(request_aoi_3, auth)
    poll_for_success(order_url_aoi_3, auth)
    time.sleep(30)  # Avoid 423 exception for too many requests 
    downloaded_files_aoi_3 = download_order(order_url_aoi_3, auth, False, 'imagery/planet/aoi-3')

else:
    print('Products already ordered')

Running search for a maximum of: 35 assets
Total estimated cost to quota: 7,004 sqkm sqkm
Total estimated cost to quota if clipped: 1 sqkm
<Response [202]>
0d255e15-79e3-4a92-b984-ee8296efc543
queued
running
running
running
running
running
running
running
success
<Response [200]>
5 items to download
downloading 0d255e15-79e3-4a92-b984-ee8296efc543/PSScene4Band/20170923_090203_0f24_3B_AnalyticMS_metadata_clip.xml to imagery/planet/aoi-3/0d255e15-79e3-4a92-b984-ee8296efc543/PSScene4Band/20170923_090203_0f24_3B_AnalyticMS_metadata_clip.xml
downloading 0d255e15-79e3-4a92-b984-ee8296efc543/PSScene4Band/20170923_090203_0f24_3B_AnalyticMS_DN_udm_clip.tif to imagery/planet/aoi-3/0d255e15-79e3-4a92-b984-ee8296efc543/PSScene4Band/20170923_090203_0f24_3B_AnalyticMS_DN_udm_clip.tif
downloading 0d255e15-79e3-4a92-b984-ee8296efc543/PSScene4Band/20170923_090203_0f24_3B_AnalyticMS_SR_clip.tif to imagery/planet/aoi-3/0d255e15-79e3-4a92-b984-ee8296efc543/PSScene4Band/20170923_090203_0f24_3B_AnalyticMS_S

We have now finished downloading the imagery. If you inspect it, you will see that each downloaded image consists of a 4 band image whenever the raster overlaps with one of the envelope boxes in the AOI, and is otherwise missing. In the next notebook, we will stitch each of the images together, calculate VIs, calculate zonal statistics, and then compare the output to Sentinel-2 imagery from a similar date. 